In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [103]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [36]:
featuresPath=os.listdir('drive/MyDrive/features')
print(featuresPath)

['2features.csv', 'countofWords.csv', 'tfidfTop10.csv', 'bertFeatureVectors.csv', 'word2vecFeatures100dims.csv', 'doc2vec.csv']


In [38]:
# tf idf features matrix
tfIDF=pd.read_csv('drive/MyDrive/features/tfidfTop10.csv')
# polarity features matrix
polarityFeatures=pd.read_csv('drive/MyDrive/features/countofWords.csv')
# subj + puncs 
twoCombined=pd.read_csv('drive/MyDrive/features/2features.csv')
# seperating punctuation feature
puncFeatures=twoCombined.iloc[:,3:6]
# sparing subjectivity score
subjectivityScore=twoCombined.Subjectivity_Score
# bert Features matrix
bertVectors=pd.read_csv('drive/MyDrive/features/bertFeatureVectors.csv')
# word2Vec
word2Vec=pd.read_csv('drive/MyDrive/features/word2vecFeatures100dims.csv')
# doc2Vec
doc2Vec=pd.read_csv('drive/MyDrive/features/doc2vec.csv')

In [43]:
namesDict={}
for i in range(0,len(word2Vec.columns)):
  namesDict[word2Vec.columns[i]]= 'word2Vec_'+ word2Vec.columns[i]

In [45]:
word2Vec.rename(columns = namesDict, inplace = True)

In [48]:
# droping class labels from all of them
bertVectors.drop('Sentiments',axis=1)
doc2Vec.drop('Sentiments',axis=1)
polarityFeatures.drop('Sentiments',axis=1)
tfIDF.drop('Sentiments',axis=1)

,hotel,stay,staff,great,breakfast,clean,night,nice,location,walk
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
41023,0,0,0,0,0,0,0,0,0,0
41024,0,0,0,0,0,0,0,0,0,0
41025,0,0,0,0,0,0,0,0,0,0
41026,0,0,0,0,0,0,0,0,0,0


In [59]:
# merging all features
allFeatures=pd.concat([subjectivityScore,tfIDF,word2Vec,polarityFeatures,doc2Vec,puncFeatures,bertVectors],axis=1)

In [81]:
# allFeatures.insert(len(allFeatures.columns),'Sentiments',twoCombined['Sentiments'])
allFeatures.sample(frac=1).reset_index(drop=True)
allFeatures.head(10)

,Subjectivity_Score,hotel,stay,staff,great,breakfast,clean,night,nice,location,...,503,504,505,506,507,508,509,510,511,Sentiments
0,0.660000,0,0,0,0,0,0,0,0,0,...,6.111171,0.470527,25.531233,-38.840145,-8.177046,-10.366693,23.783344,54.813313,52.485943,1
1,0.750000,0,0,0,0,0,0,0,0,0,...,-15.305029,-16.290522,25.800335,-31.154448,24.325838,2.358032,12.143825,64.162710,47.817444,1
2,0.487963,0,0,0,0,0,0,0,0,0,...,-32.100662,-21.980486,35.545330,-17.454865,-11.807137,8.905584,32.914500,35.619366,48.572678,1
3,0.476190,0,0,0,0,0,0,0,0,0,...,0.418124,-31.429620,21.996190,-14.593822,19.299784,26.459166,52.918346,36.160220,60.949040,1
4,0.749320,0,0,0,0,0,0,0,0,0,...,30.136467,-5.655626,-1.015279,-25.154745,-6.560827,56.230213,46.088623,29.860981,18.640345,1
5,0.633333,0,0,0,0,0,0,0,0,0,...,-70.755590,-18.507032,16.916576,-36.055645,-4.592214,27.491219,-19.323635,-5.725876,-1.876659,1
6,0.554304,0,0,0,0,0,0,0,0,0,...,5.502983,7.601984,43.793518,-40.898937,12.110632,-11.307594,6.190011,41.204155,47.116170,1
7,0.547959,0,0,0,0,0,0,0,0,0,...,-15.095938,5.988987,30.353329,-32.748055,8.251566,-8.298467,62.668854,46.042732,52.964375,1
8,0.614035,0,0,0,0,0,0,0,0,0,...,-17.237345,-12.258137,25.655262,-13.112321,38.684193,-38.002365,40.519432,29.747780,42.820683,1
9,0.476111,0,0,0,0,0,0,0,0,0,...,-40.942990,-24.376790,22.716091,-2.249550,-26.449335,-5.334550,33.223510,29.030748,30.990149,1


In [82]:
allFeatures.Sentiments

0        1
1        1
2        1
3        1
4        1
        ..
41023    0
41024    0
41025    0
41026    0
41027    0
Name: Sentiments, Length: 41028, dtype: int64

In [78]:
allFeatures=allFeatures.drop('Sentiments',axis=1)

In [94]:
# random forest model
X=allFeatures.drop('Sentiments',axis=1)
y=allFeatures.Sentiments
# # defining classifier
clf=RandomForestClassifier()
# # 10 fold cross validation
scores = cross_val_score(clf, X, y, cv=10)

In [91]:
X=allFeatures.drop('Sentiments',axis=1)
y=allFeatures.Sentiments
# defining classifier
clf=svm.SVC()
# 10 fold cross validation
SVMscores = cross_val_score(clf, X, y, cv=10)

In [93]:
print('svm results-->',np.mean(SVMscores))

svm results--> 0.8136631780411119


In [95]:
import numpy as np
print('random forest results ->',np.mean(scores))

random forest results -> 0.9748464247004813


In [ ]:
# hold out method for all features

In [ ]:
# random forest training on all features and saaving the model

In [105]:
X=allFeatures.drop('Sentiments',axis=1)
y=allFeatures.Sentiments
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [110]:
clf = RandomForestClassifier()

In [111]:
clf.fit(X_train, y_train)

RandomForestClassifier()

In [114]:
res=clf.predict(X_test)

In [115]:
accuracy_score(y_test, res)

0.973677796734097

In [116]:
# saving trained model
import pickle
pickle.dump(clf, open('forest_model_allFeatures', 'wb'))

In [123]:
# loading saved model
r_model = pickle.load(open('forest_model_allFeatures', 'rb'))

In [119]:
res=model.predict(X_test)
accuracy_score(y_test, res)

0.973677796734097

In [ ]:
# svm training on all features and saving model

In [120]:
clf = svm.SVC()
clf.fit(X_train, y_train)
res=clf.predict(X_test)
accuracy_score(y_test, res)

0.821593955642213

In [121]:
# saving trained model
import pickle
pickle.dump(clf, open('svm_model_allFeatures', 'wb'))

In [126]:
# adding predicted labels in features matrix for random forest
predictions=r_model.predict(X)
allFeatures.insert(len(allFeatures.columns),'RandomForest_Predictions',predictions)
allFeatures.head()

,Subjectivity_Score,hotel,stay,staff,great,breakfast,clean,night,nice,location,...,504,505,506,507,508,509,510,511,Sentiments,RandomForest_Predictions
0,0.660000,0,0,0,0,0,0,0,0,0,...,0.470527,25.531233,-38.840145,-8.177046,-10.366693,23.783344,54.813313,52.485943,1,1
1,0.750000,0,0,0,0,0,0,0,0,0,...,-16.290522,25.800335,-31.154448,24.325838,2.358032,12.143825,64.162710,47.817444,1,1
2,0.487963,0,0,0,0,0,0,0,0,0,...,-21.980486,35.545330,-17.454865,-11.807137,8.905584,32.914500,35.619366,48.572678,1,1
3,0.476190,0,0,0,0,0,0,0,0,0,...,-31.429620,21.996190,-14.593822,19.299784,26.459166,52.918346,36.160220,60.949040,1,1
4,0.749320,0,0,0,0,0,0,0,0,0,...,-5.655626,-1.015279,-25.154745,-6.560827,56.230213,46.088623,29.860981,18.640345,1,1


In [128]:
# adding predicted labels in features matrix for svm
# load model
# loading saved model
svm_model = pickle.load(open('svm_model_allFeatures', 'rb'))
# predict
predictions=svm_model.predict(X)

In [128]:
allFeatures.insert(len(allFeatures.columns),'SVM_Predictions',predictions)
allFeatures.head()

,Subjectivity_Score,hotel,stay,staff,great,breakfast,clean,night,nice,location,...,505,506,507,508,509,510,511,Sentiments,RandomForest_Predictions,SVM_Predictions
0,0.660000,0,0,0,0,0,0,0,0,0,...,25.531233,-38.840145,-8.177046,-10.366693,23.783344,54.813313,52.485943,1,1,1
1,0.750000,0,0,0,0,0,0,0,0,0,...,25.800335,-31.154448,24.325838,2.358032,12.143825,64.162710,47.817444,1,1,1
2,0.487963,0,0,0,0,0,0,0,0,0,...,35.545330,-17.454865,-11.807137,8.905584,32.914500,35.619366,48.572678,1,1,1
3,0.476190,0,0,0,0,0,0,0,0,0,...,21.996190,-14.593822,19.299784,26.459166,52.918346,36.160220,60.949040,1,1,1
4,0.749320,0,0,0,0,0,0,0,0,0,...,-1.015279,-25.154745,-6.560827,56.230213,46.088623,29.860981,18.640345,1,1,1


In [129]:
# saving the file as csv
allFeatures.to_csv('drive/MyDrive/features/featureswithpredictions.csv')